# Strands エージェントと AgentCore Memory（短期メモリ）


## はじめに

このチュートリアルでは、Strands エージェントと AgentCore の**短期メモリ**（Raw イベント）を使用して**パーソナルエージェント**を構築する方法を紹介します。エージェントは `get_last_k_turns` を使用してセッション内の最近の会話を記憶し、ユーザーが戻ってきた際にシームレスに会話を継続できます。


### チュートリアルの詳細

| 情報               | 詳細                                                                            |
|:-------------------|:--------------------------------------------------------------------------------|
| チュートリアルタイプ | 短期会話メモリ                                                                  |
| エージェントタイプ   | パーソナルエージェント                                                          |
| エージェントフレームワーク | Strands Agents                                                                 |
| LLM モデル          | Anthropic Claude Haiku 4.5                                                     |
| チュートリアルコンポーネント | AgentCore 短期メモリ、AgentInitializedEvent と MessageAddedEvent フック          |
| 難易度              | 初級                                                                            |

学習内容：
- 会話の継続性のための短期メモリの使用
- 直近 K ターンの会話履歴の取得
- リアルタイム情報取得のための Web 検索ツール
- 会話履歴を持つエージェントの初期化

## アーキテクチャ
<div style="text-align:left">
    <img src="architecture.png" width="65%" />
</div>

## 前提条件

- Python 3.10以上
- AgentCore Memory 権限を持つ AWS 認証情報
- AgentCore Memory ロール ARN
- Amazon Bedrock モデルへのアクセス

環境のセットアップから始めましょう！

## ステップ 1: セットアップとインポート

In [ ]:
!pip install -qr requirements.txt
!pip install duckduckgo-search

In [ ]:
import logging
from datetime import datetime

# ロギングのセットアップ
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("personal-agent")

In [ ]:
# インポート
import os
from strands import Agent, tool
from strands.hooks import AgentInitializedEvent, HookProvider, HookRegistry, MessageAddedEvent
from bedrock_agentcore.memory import MemoryClient

# 設定
REGION = os.getenv('AWS_REGION', 'us-west-2') # エージェントの AWS リージョン
ACTOR_ID = "user_123" # 任意の一意識別子（エージェント ID、ユーザー ID など）
SESSION_ID = "personal_session_001" # 一意のセッション識別子

## ステップ 2: Web 検索ツール

まず、エージェント用のシンプルな Web 検索ツールを作成しましょう。

In [ ]:
from ddgs.exceptions import DDGSException, RatelimitException
from ddgs import DDGS

@tool
def websearch(keywords: str, region: str = "us-en", max_results: int = 5) -> str:
    """最新情報を取得するために Web を検索します。
    
    Args:
        keywords (str): 検索クエリのキーワード。
        region (str): 検索リージョン: wt-wt, us-en, uk-en, ru-ru など。
        max_results (int | None): 返す結果の最大数。
    Returns:
        検索結果の辞書リスト。
    
    """
    try:
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "結果が見つかりませんでした。"
    except RatelimitException:
        return "レート制限に達しました。後でもう一度お試しください。"
    except DDGSException as e:
        return f"検索エラー: {e}"
    except Exception as e:
        return f"検索エラー: {str(e)}"

logger.info("✅ Web 検索ツール準備完了")

## ステップ 3: Memory リソースの作成
短期メモリの場合、ストラテジーなしでメモリリソースを作成します。これにより、`get_last_k_turns` で取得できる生の会話ターンが保存されます。


In [ ]:
from botocore.exceptions import ClientError

# Memory Client の初期化
client = MemoryClient(region_name=REGION)
memory_name = "PersonalAgentMemory"

try:
    # ストラテジーなしでメモリリソースを作成（短期メモリのみアクセス）
    memory = client.create_memory_and_wait(
        name=memory_name,
        strategies=[],  # 短期メモリ用のストラテジーなし
        description="Short-term memory for personal agent",
        event_expiry_days=7, # 短期メモリの保持期間。最大365日まで可能。
    )
    memory_id = memory['id']
    logger.info(f"✅ メモリを作成しました: {memory_id}")
except ClientError as e:
    logger.info(f"❌ エラー: {e}")
    if e.response['Error']['Code'] == 'ValidationException' and "already exists" in str(e):
        # メモリが既に存在する場合は、その ID を取得
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['id'].startswith(memory_name)), None)
        logger.info(f"メモリは既に存在します。既存のメモリ ID を使用: {memory_id}")
except Exception as e:
    # メモリ作成中のエラーを表示
    logger.error(f"❌ エラー: {e}")
    import traceback
    traceback.print_exc()
    # エラー時のクリーンアップ - 部分的に作成されたメモリを削除
    if memory_id:
        try:
            client.delete_memory_and_wait(memory_id=memory_id)
            logger.info(f"メモリをクリーンアップしました: {memory_id}")
        except Exception as cleanup_error:
            logger.error(f"メモリのクリーンアップに失敗しました: {cleanup_error}")

## ステップ 4: Memory フック

このステップでは、メモリ操作を自動化するカスタム `MemoryHookProvider` クラスを定義します。フックは、エージェントの実行ライフサイクルの特定のポイントで実行される特別な関数です。作成するメモリフックには2つの主要な機能があります：
1. **最近の会話をロード**: エージェントが初期化されたときに自動的に最近の会話履歴をロードするための `AgentInitializedEvent` フック。
2. **最新のメッセージを保存**: 新しい会話メッセージを保存。

これにより、手動管理なしでシームレスなメモリ体験が実現します。

In [ ]:
class MemoryHookProvider(HookProvider):
    def __init__(self, memory_client: MemoryClient, memory_id: str):
        self.memory_client = memory_client
        self.memory_id = memory_id
    
    def on_agent_initialized(self, event: AgentInitializedEvent):
        """エージェント開始時に最近の会話履歴をロード"""
        try:
            # エージェント状態からセッション情報を取得
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")
            
            if not actor_id or not session_id:
                logger.warning("エージェント状態に actor_id または session_id がありません")
                return
            
            # メモリから直近5ターンの会話を取得
            recent_turns = self.memory_client.get_last_k_turns(
                memory_id=self.memory_id,
                actor_id=actor_id,
                session_id=session_id,
                k=5
            )
            
            if recent_turns:
                # コンテキスト用に会話履歴をフォーマット
                context_messages = []
                for turn in recent_turns:
                    for message in turn:
                        role = message['role']
                        content = message['content']['text']
                        context_messages.append(f"{role}: {content}")
                
                context = "\n".join(context_messages)
                # エージェントのシステムプロンプトにコンテキストを追加
                event.agent.system_prompt += f"\n\n最近の会話:\n{context}"
                logger.info(f"✅ {len(recent_turns)} ターンの会話をロードしました")
                
        except Exception as e:
            logger.error(f"メモリロードエラー: {e}")
    
    def on_message_added(self, event: MessageAddedEvent):
        """メッセージをメモリに保存"""
        messages = event.agent.messages
        try:
            # エージェント状態からセッション情報を取得
            actor_id = event.agent.state.get("actor_id")
            session_id = event.agent.state.get("session_id")

            if messages[-1]["content"][0].get("text"):
                self.memory_client.create_event(
                    memory_id=self.memory_id,
                    actor_id=actor_id,
                    session_id=session_id,
                    messages=[(messages[-1]["content"][0]["text"], messages[-1]["role"])]
                )
        except Exception as e:
            logger.error(f"メモリ保存エラー: {e}")
    
    def register_hooks(self, registry: HookRegistry):
        # メモリフックを登録
        registry.add_callback(MessageAddedEvent, self.on_message_added)
        registry.add_callback(AgentInitializedEvent, self.on_agent_initialized)

## ステップ 5: Web 検索機能付きパーソナルエージェントの作成

In [ ]:
def create_personal_agent():
    """メモリと Web 検索機能を持つパーソナルエージェントを作成"""
    agent = Agent(
        name="PersonalAssistant",
        model="global.anthropic.claude-haiku-4-5-20251001-v1:0",  # または希望のモデル
        system_prompt=f"""あなたは Web 検索機能を持つ親切なパーソナルアシスタントです。
        
        あなたができること：
        - 一般的な質問と情報検索
        - 最新情報の Web 検索
        - 個人的なタスク管理
        
        最新情報が必要な場合は、websearch 関数を使用してください。
        今日の日付: {datetime.today().strftime('%Y-%m-%d')}
        フレンドリーかつプロフェッショナルに対応してください。""",
        hooks=[MemoryHookProvider(client, memory_id)],
        tools=[websearch],
        state={"actor_id": ACTOR_ID, "session_id": SESSION_ID}
    )
    return agent

# エージェントを作成
agent = create_personal_agent()
logger.info("✅ メモリと Web 検索機能を持つパーソナルエージェントを作成しました")

#### おめでとうございます！エージェントの準備ができました！ :) 
## エージェントをテストしましょう

In [ ]:
# メモリ付き会話テスト
print("=== 最初の会話 ===")
print(f"ユーザー: 私の名前はアレックスで、AI について学ぶことに興味があります。")
print(f"エージェント: ", end="")
agent("My name is Alex and I'm interested in learning about AI.")

In [ ]:
print(f"ユーザー: 2025年の最新AI トレンドを検索してもらえますか？")
print(f"エージェント: ", end="")
agent("Can you search for the latest AI trends in 2025?")

In [ ]:
print(f"ユーザー: 特に機械学習の応用に興味があります。")
print(f"エージェント: ", end="")
agent("I'm particularly interested in machine learning applications.")

## メモリの継続性をテスト

メモリシステムが正しく動作しているかテストするために、エージェントの新しいインスタンスを作成し、以前に保存された情報にアクセスできるか確認します：

In [ ]:
# 新しいエージェントインスタンスを作成（ユーザーが戻ってきた状況をシミュレート）
print("=== ユーザーが戻ってきました - 新しいセッション ===")
new_agent = create_personal_agent()

# メモリの継続性をテスト
print(f"ユーザー: 私の名前は何でしたっけ？")
print(f"エージェント: ", end="")
new_agent("What was my name again?")

print(f"ユーザー: 機械学習についてもっと情報を検索してもらえますか？")
print(f"エージェント: ", end="")
new_agent("Can you search for more information about machine learning?")

## 保存されたメモリの確認

In [ ]:
# メモリに保存されている内容を確認
print("=== メモリの内容 ===")
recent_turns = client.get_last_k_turns(
    memory_id=memory_id,
    actor_id=ACTOR_ID,
    session_id=SESSION_ID,
    k=3 # より多くまたは少ないターンを表示するには k を調整
)

for i, turn in enumerate(recent_turns, 1):
    print(f"ターン {i}:")
    for message in turn:
        role = message['role']
        content = message['content']['text'][:100] + "..." if len(message['content']['text']) > 100 else message['content']['text']
        print(f"  {role}: {content}")
    print()

## まとめ

このチュートリアルでは、パーソナルエージェントの構築方法を紹介しました。学習した内容：

- ストラテジーなしのメモリリソースの作成
- 会話履歴取得のための `get_last_k_turns` の使用
- エージェントへの Web 検索機能の追加
- コンテキストロード用のメモリフックの実装

**次のステップ：**
- より高度なツールの追加
- 長期メモリストラテジーの実装
- 複数ソースでの検索機能の強化

## クリーンアップ（オプション）

In [ ]:
# メモリリソースを削除するにはコメントを解除
# client.delete_memory_and_wait(memory_id)
# logger.info(f"✅ メモリを削除しました: {memory_id}")